### Generate videos from the PolarVid Dataset
Zuerl M, Dirauf R, Koeferl F, Steinlein N, Sueskind J, Zanca D, Brehm I, Fersen LV, Eskofier B. PolarBearVidID: A Video-Based Re-Identification Benchmark Dataset for Polar Bears. Animals (Basel). 2023 Feb 23;13(5):801. doi: 10.3390/ani13050801. PMID: 36899661; PMCID: PMC10000026.
1. The frames are 12.5fps while the resolution of the image is 256x128
2. The directory contains the unique ids (labels) for the different polar bears identity
3. Example of the frame labels is `000C01T000F099` The first `000` contains the unique animal id, `C01T000` Contains a different angle/video of the animal while `F0__` contains the frames captured of that animal

<center>

| ID | Name      | Gender | Birth | Zoo         | 
|----|-----------|--------|-------|-------------|
| 0  | Vera      | Female | 2002  | Nuremberg   |
| 1  | Nanuq     | Male   | 2007  | Nuremberg   | 
| 2  | Tonja     | Female | 2009  | Berlin      | 
| 3  | Hertha    | Female | 2018  | Berlin      | 
| 4  | Nora      | Female | 2013  | Vienna      | 
| 5  | Finja     | Female | 2019  | Vienna      | 
| 6  | Larissa   | Female | 1990  | Neumünster  | 
| 7  | Vitus     | Male   | 2000  | Neumünster  | 
| 8  | Lloyd     | Male   | 2000  | Karlsruhe   | 
| 9  | Charlotte | Female | 2014  | Karlsruhe   | 
| 10 | Nana      | Female | 2019  | Hannover    | 
| 11 | Milana    | Female | 2009  | Hannover    |
| 12 | Sprinter  | Male   | 2007  | Hannover    | 

</center>

In [74]:
'''
Creating a video would be to handle the directory as a series of images and abiding by the naming convention of the images mentioned in the previous section. 
The following code snippet shows how to create a video from a series of images:
'''
import cv2
import os

# The animal names
animal_names = {
    0: 'Vera',
    1: 'Nanuq',
    2: 'Tonja',
    3: 'Hertha',
    4: 'Nora',
    5: 'Finja',
    6: 'Larissa',
    7: 'Vitus',
    8: 'Lloyd',
    9: 'Charlotte',
    10: 'Nana',
    11: 'Milana',
    12: 'Sprinter'
}

# A method to take in a particular directory and create videos from the images in that directory
def create_video_from_directory(dir_path, output_video_path):
    animal_id = os.path.basename(dir_path).split('C')[0]
    animal_name = animal_names[int(animal_id)]

    # Create the output directory if it does not exist plus animal_name
    output_video_path = os.path.join(output_video_path, animal_name)
    if not os.path.exists(output_video_path):
        os.makedirs(output_video_path)

    current_file_part = None
    count = 0
    out = None

    # List all files in the directory, sorted by name
    for file_name in sorted(os.listdir(dir_path)):
        # Split the file name as required to extract the relevant part
        file_part = os.path.splitext(file_name)[0].split('F')[0].split('C')[1]

        # If the file part changes, update the current_file_part and create a new video
        if file_part != current_file_part:
            if out is not None:
                out.release()

            current_file_part = file_part
            video_file_path = os.path.join(output_video_path, f"{animal_name}_video_{count}.avi")

            # Read the first image to determine the frame size
            first_img_path = os.path.join(dir_path, file_name)
            first_img = cv2.imread(first_img_path)
            if first_img is None:
                print(f"Error reading image {first_img_path}")
                continue

            frame_height, frame_width = first_img.shape[:2]
            fps = 12.5
            out = cv2.VideoWriter(video_file_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width, frame_height))
            print(f'Creating video: {video_file_path}')
            count += 1

        # Read the image and write it to the video
        img_path = os.path.join(dir_path, file_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error reading image {img_path}")
            continue
        out.write(img)

    # Release the video writer object
    if out is not None:
        out.release()


In [82]:
dir_path = 'run-gsp\data\PolarBearVid'
output_video_path = 'run-gsp\data\PolarBearVideos'
for i in range(13):
    dir_path2 = os.path.join(dir_path, f'{i:03}')  # Zero-padded directory names from 000 to 012
    if os.path.exists(dir_path2):
        print(f'Processing directory: {dir_path2}')
        create_video_from_directory(dir_path2, output_video_path)